# 在 Hugging Face Spaces 上设置 Phoenix 观察性仪表板以进行 LLM 应用程序追踪

_作者：[Andrew Reed](https://huggingface.co/andrewrreed)_

[Phoenix](https://docs.arize.com/phoenix) 是由 [Arize AI](https://arize.com/) 开发的开源观察性库，旨在进行实验、评估和故障排除。它允许 AI 工程师和数据科学家快速可视化他们的数据、评估性能、追踪问题，并导出数据以进行改进。

<div style="display: flex; justify-content: space-between; gap: 20px; margin: 20px 0;">
    <div style="width: 32%; text-align: center; display: flex; flex-direction: column; align-items: center; justify-content: flex-start;">
        <img src="https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/phoenix-tracing.png" style="width: 100%; height: 200px; object-fit: contain;"/>
        <p style="margin-top: 10px; margin-bottom: 0;">追踪</p>
    </div>
    <div style="width: 32%; text-align: center; display: flex; flex-direction: column; align-items: center; justify-content: flex-start;">
        <img src="https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/phoenix-datasets.png" style="width: 100%; height: 200px; object-fit: contain;"/>
        <p style="margin-top: 10px; margin-bottom: 0;">数据集</p>
    </div>
    <div style="width: 32%; text-align: center; display: flex; flex-direction: column; align-items: center; justify-content: flex-start;">
        <img src="https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/phoenix-experiments.png" style="width: 100%; height: 200px; object-fit: contain;"/>
        <p style="margin-top: 10px; margin-bottom: 0;">实验</p>
    </div>
</div>

在本 Notebook 中，我们将展示如何在 [Hugging Face Spaces](https://huggingface.co/spaces) 上部署 Phoenix 观察性仪表板，并配置它以自动追踪 LLM 调用，从而全面了解 LLM 应用程序的内部工作原理。

## 第一步：在 Hugging Face Spaces 上部署 Phoenix

虽然 Phoenix 提供了一个用于本地开发的 [Notebook 高级选项](https://docs.arize.com/phoenix/deployment/environments#notebooks)，它也可以通过 [Docker 作为独立仪表板部署](https://docs.arize.com/phoenix/deployment/environments#container)。长时间运行的托管仪表板是提供 LLM 应用行为集中视图的好方法，并且有助于团队协作。Hugging Face Spaces 提供了一种简单的方式来托管 ML 应用，并支持可选的持久存储，且它对自定义 Docker 镜像的支持使其成为托管 Phoenix 的理想平台——我们来看看它是如何工作的！

首先，我们将 [复制示例空间](https://huggingface.co/spaces/andrewrreed/phoenix-arize-observability-demo?duplicate=true)

![](https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/duplicate.png)

我们可以将空间配置为私有或公开，并且它可以位于我们的用户命名空间或组织命名空间中。我们可以使用默认的免费 CPU 配置，并且重要的是，我们可以指定要为该空间附加一个持久磁盘。

> [!TIP] 为了使追踪数据在 Space 重启后保持持久，我们 _必须_ 配置持久磁盘，否则所有数据将在 Space 重启时丢失。配置持久磁盘是付费功能，并会产生 Space 存续期间的费用。在本例中，我们将使用小型 - 20GB 磁盘选项，每小时 $0.01。

点击“复制空间”后，Docker 镜像将开始构建。这将需要几分钟时间完成，之后我们将看到一个空的 Phoenix 仪表板。

![](https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/empty-dashboard.png)

## 第二步：配置应用追踪

现在我们已经有了一个运行中的 Phoenix 仪表板，可以使用 [OpenTelemetry TracerProvider](https://docs.arize.com/phoenix/quickstart#connect-your-app-to-phoenix) 配置应用程序，以自动追踪 LLM 调用。在这个例子中，我们将使用 OpenAI 客户端库来给应用程序加上监控，并追踪从 `openai` Python 包发出的 LLM 调用，这些调用会被发送到 [Hugging Face 的 Serverless 推理 API](https://huggingface.co/docs/api-inference/en/index) 上运行的 LLM。

> [!TIP] Phoenix 支持对 [多种 LLM 框架](https://docs.arize.com/phoenix/tracing/integrations-tracing) 进行追踪，包括 LangChain、LlamaIndex、AWS Bedrock 等。

首先，我们需要安装必要的库：

In [ ]:
!pip install -q arize-phoenix arize-phoenix-otel openinference-instrumentation-openai openai huggingface-hub

接下来，我们将使用 `huggingface_hub` 库登录到 Hugging Face。这将允许我们生成所需的身份验证，以便为我们的 Space 和 Serverless 推理 API 提供认证。请确保用于身份验证的 Hugging Face 令牌具有正确的权限，以便访问存放你 Space 的组织。

In [ ]:
from huggingface_hub import interpreter_login

interpreter_login()

现在，我们可以将 [Phoenix 客户端配置](https://docs.arize.com/phoenix/deployment/configuration#client-configuration) 为连接到我们运行中的 Phoenix 仪表板：

1. 注册 Phoenix 追踪提供者：
    - 指定我们选择的 `project_name`
    - 将 `endpoint` 设置为我们 Space 的主机名（可以通过仪表板 UI 中的“设置”标签找到，如下图所示）
    - 将 `headers` 设置为访问 Space 所需的 Hugging Face 头信息
2. 在我们的应用代码中，使用 OpenAI 追踪提供者进行监控

![](https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/settings.png)

In [1]:
from phoenix.otel import register
from huggingface_hub.utils import build_hf_headers
from openinference.instrumentation.openai import OpenAIInstrumentor

# 1. Register the Phoenix tracer provider
tracer_provider = register(
    project_name="test",
    endpoint="https://andrewrreed-phoenix-arize-observability-demo.hf.space"
    + "/v1/traces",
    headers=build_hf_headers(),
)

# 2. Instrument our application code to use the OpenAI tracer provider
OpenAIInstrumentor().instrument(tracer_provider=tracer_provider)

🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: test
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: https://andrewrreed-phoenix-arize-observability-demo.hf.space/v1/traces
|  Transport: HTTP
|  Transport Headers: {'user-agent': '****', 'authorization': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



## 第三步：发起调用并在 Phoenix 仪表板中查看追踪

现在，我们可以调用 LLM 并在 Phoenix 仪表板中查看追踪。我们使用 OpenAI 客户端向 Hugging Face 的 Serverless 推理 API 发起调用，该 API 已经配置为与 Phoenix 配合工作。在这个例子中，我们使用的是 `meta-llama/Llama-3.1-8B-Instruct` 模型。

In [2]:
from openai import OpenAI
from huggingface_hub import get_token

client = OpenAI(
    base_url="https://api-inference.huggingface.co/v1/",
    api_key=get_token(),
)

messages = [{"role": "user", "content": "What does a llama do for fun?"}]

response = client.chat.completions.create(
    model="meta-llama/Llama-3.1-8B-Instruct",
    messages=messages,
    max_tokens=500,
)

print(response.choices[0].message.content)

Llamas are intelligent and social animals, and they do have ways to entertain themselves and have fun. While we can't directly ask a llama about its personal preferences, we can observe their natural behaviors and make some educated guesses. Here are some things that might bring a llama joy and excitement:

1. **Socializing**: Llamas are herd animals and they love to interact with each other. They'll often engage in play-fighting, neck-wrestling, and other activities to establish dominance and strengthen social bonds. When llamas have a strong social network, it can make them feel happy and content.
2. **Exploring new environments**: Llamas are naturally curious creatures, and they love to explore new surroundings. They'll often investigate their surroundings, sniffing and investigating new sights, sounds, and smells.
3. **Playing with toys**: While llamas don't need expensive toys, they do enjoy playing with objects that stimulate their natural behaviors. For example, a ball or a toy 

如果我们返回到 Phoenix 仪表板，我们可以看到从 LLM 调用捕获并显示的追踪信息！如果你在配置 Space 时使用了持久磁盘，那么每次重启 Space 时，所有的追踪信息都会被保存。

![](https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/test-trace.png)

## 奖励部分：使用 CrewAI 追踪多代理应用程序

观察性的真正优势在于能够追踪和检查复杂的 LLM 工作流。在这个例子中，我们将安装并使用 [CrewAI](https://www.crewai.com/) 来追踪一个多智能体应用程序。

> [!TIP] `openinference-instrumentation-crewai` 包目前要求 Python 3.10 或更高版本。安装 `crewai` 库后，你可能需要重启 Notebook 内核以避免出现错误。

In [ ]:
!pip install -q openinference-instrumentation-crewai crewai crewai-tools

和之前一样，我们将注册 Phoenix 追踪提供者并给应用代码加上监控，但这次我们还需要取消现有 OpenAI 追踪提供者的监控，以避免冲突。

In [3]:
from opentelemetry import trace
from openinference.instrumentation.crewai import CrewAIInstrumentor

# 0. Uninstrument existing tracer provider and clear the global tracer provider
OpenAIInstrumentor().uninstrument()
if trace.get_tracer_provider():
    trace.get_tracer_provider().shutdown()
    trace._TRACER_PROVIDER = None  # Reset the global tracer provider

# 1. Register the Phoenix tracer provider
tracer_provider = register(
    project_name="crewai",
    endpoint="https://andrewrreed-phoenix-arize-observability-demo.hf.space"
    + "/v1/traces",
    headers=build_hf_headers(),
)

# 2. Instrument our application code to use the OpenAI tracer provider
CrewAIInstrumentor().instrument(tracer_provider=tracer_provider)

Overriding of current TracerProvider is not allowed


🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: crewai
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: https://andrewrreed-phoenix-arize-observability-demo.hf.space/v1/traces
|  Transport: HTTP
|  Transport Headers: {'user-agent': '****', 'authorization': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



现在，我们将使用 CrewAI 定义一个多智能体应用程序，研究并撰写一篇关于观察性和追踪在 LLM 应用中的重要性的博客。

> [!TIP] 这个例子借鉴并修改自 [此处](https://docs.arize.com/phoenix/tracing/integrations-tracing/crewai)。

In [5]:
import os
from huggingface_hub import get_token
from crewai_tools import SerperDevTool
from crewai import LLM, Agent, Task, Crew, Process

# Define our LLM using HF's Serverless Inference API
llm = LLM(
    model="huggingface/meta-llama/Llama-3.1-8B-Instruct",
    api_key=get_token(),
    max_tokens=1024,
)

# Define a tool for searching the web
os.environ["SERPER_API_KEY"] = (
    "YOUR_SERPER_API_KEY"  # must set this value in your environment
)
search_tool = SerperDevTool()

# Define your agents with roles and goals
researcher = Agent(
    role="Researcher",
    goal="Conduct thorough research on up to date trends around a given topic.",
    backstory="""You work at a leading tech think tank. You have a knack for dissecting complex data and presenting actionable insights.""",
    verbose=True,
    allow_delegation=False,
    tools=[search_tool],
    llm=llm,
    max_iter=1,
)
writer = Agent(
    role="Technical Writer",
    goal="Craft compelling content on a given topic.",
    backstory="""You are a technical writer with a knack for crafting engaging and informative content.""",
    llm=llm,
    verbose=True,
    allow_delegation=False,
    max_iter=1,
)

# Create tasks for your agents
task1 = Task(
    description="""Conduct comprehensive research and analysis of the importance of observability and tracing in LLM applications.
  Identify key trends, breakthrough technologies, and potential industry impacts.""",
    expected_output="Full analysis report in bullet points",
    agent=researcher,
)

task2 = Task(
    description="""Using the insights provided, develop an engaging blog
  post that highlights the importance of observability and tracing in LLM applications.
  Your post should be informative yet accessible, catering to a tech-savvy audience.
  Make it sound cool, avoid complex words so it doesn't sound like AI.""",
    expected_output="Blog post of at least 3 paragraphs",
    agent=writer,
)

# Instantiate your crew with a sequential process
crew = Crew(
    agents=[researcher, writer],
    tasks=[task1, task2],
    verbose=True,
    process=Process.sequential,
)

# Get your crew to work!
result = crew.kickoff()

print("------------ FINAL RESULT ------------")
print(result)

2024-10-31 09:10:49,986 - 8143739712 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Researcher
## Task: Conduct comprehensive research and analysis of the importance of observability and tracing in LLM applications.
  Identify key trends, breakthrough technologies, and potential industry impacts.


# Agent: Researcher
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"importance of observability and tracing in LLM applications\"}"
## Tool Output: 

Search results: Title: LLM Observability: The 5 Key Pillars for Monitoring Large Language ...
Link: https://arize.com/blog-course/large-language-model-monitoring-observability/
Snippet: Why leveraging the five pillars of LLM observability is essential for ensuring performance, reliability, and seamless LLM applications.
---
Title: Observability of LLM Applications: Exploration and Practice from the ...
Link: https://www.alibabacloud.com/blog/observability-of-llm-applications-exploration-and-practice-from-the-perspective-of-trace_601604
Snippet: This article clarifies the technical challenges o

2024-10-31 09:11:21,244 - 8143739712 - __init__.py-__init__:100 - WARNING: Invalid type TaskOutput for attribute 'output.value' value. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or a sequence of those types




# Agent: Researcher
## Final Answer: 
**Comprehensive Research and Analysis Report: Importance of Observability and Tracing in LLM Applications**

**Introduction**

Large Language Models (LLMs) have revolutionized the field of natural language processing, enabling applications such as language translation, text summarization, and conversational AI. However, as LLMs become increasingly complex and widespread, ensuring their performance, reliability, and security has become a significant challenge. Observability and tracing are crucial components in addressing this challenge, enabling developers to monitor, debug, and optimize LLM applications. This report provides an in-depth analysis of the importance of observability and tracing in LLM applications, highlighting key trends, breakthrough technologies, and potential industry impacts.

**Key Trends:**

* **Increased Adoption of Cloud-Native Observability Tools**: Cloud-native observability tools, such as OpenTelemetry and Signoz, are g

2024-10-31 09:11:39,715 - 8143739712 - __init__.py-__init__:100 - WARNING: Invalid type TaskOutput for attribute 'output.value' value. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or a sequence of those types




# Agent: Technical Writer
## Final Answer: 
**Unlocking the Power of Observability and Tracing in LLM Applications**

As Large Language Models (LLMs) continue to revolutionize the field of natural language processing, ensuring their performance, reliability, and security has become a top priority for developers. With the increasing complexity of LLMs, it's essential to have a robust monitoring and debugging system in place to identify and address issues before they impact users. This is where observability and tracing come into play – two critical components that enable developers to monitor, debug, and optimize LLM applications.

**The Importance of Observability and Tracing in LLM Applications**

Observability and tracing are not just buzzwords; they're essential tools for any LLM developer. By implementing observability and tracing, developers can gain real-time insights into their application's performance and behavior, allowing them to identify bottlenecks, optimize resource uti

返回 Phoenix 仪表板后，我们可以在新项目 "crewai" 中看到来自我们多智能体应用程序的追踪信息！

![](https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/crew-ai-trace.png)